In [1]:
! pip install pmdarima

You should consider upgrading via the 'C:\Users\Ruben\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import pmdarima as pmd
import numpy as np
from pmdarima.model_selection import train_test_split

In [11]:
dataset_year_pesca = pd.read_csv('./data/pesca_annuale.csv')
dataset_year_pesca = dataset_year_pesca.sort_values(by='year')

In [34]:
dataset_trim_pesca = pd.read_csv('./data/pesca_trimestrale.csv')
dataset_trim_pesca = dataset_trim_pesca.sort_values(by='year')

In [40]:
dataset_trim_calcio = pd.read_csv('./data/calcio_trimestrale.csv')
dataset_trim_calcio = dataset_trim_calcio.sort_values(by='year')

In [41]:
dataset_year_calcio = pd.read_csv('./data/calcio_annuale.csv')
dataset_year_calcio = dataset_year_calcio.sort_values(by='year')

In [14]:
train_year, test_year = train_test_split(dataset_year_pesca, train_size=0.8)


In [35]:
train_trim_pesca, test_trim_pesca = train_test_split(dataset_trim_pesca, train_size=0.8)


In [42]:
train_trim_calcio, test_trim_calcio = train_test_split(dataset_trim_calcio, train_size=0.8)


In [43]:
train_year_calcio, test_year_calcio = train_test_split(dataset_year_calcio, train_size=0.8)


In [50]:
train_year_calcio.head()["revenue"]

0    3365712.79
1    4234959.81
2    3876798.63
3    4552611.48
4    5435884.85
Name: revenue, dtype: float64

In [47]:
arima_model_year = pmd.auto_arima(train_year["revenue"], start_p=0, start_q=1, start_P=0, start_Q=1,max_p=4,max_q=4, max_P=4, max_Q=4, test="adf", seasonal=True,m=1,trace=True)

Performing stepwise search to minimize aic
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.03 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=229.495, Time=0.00 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=240.441, Time=0.01 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=242.512, Time=0.00 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.05 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 0.105 seconds


In [51]:
arima_model_year_calcio = pmd.auto_arima(train_year_calcio["revenue"], start_p=0, start_q=1, start_P=0, start_Q=1,max_p=4,max_q=4, max_P=4, max_Q=4, test="adf", seasonal=True,m=1,trace=True)

c:\Users\Ruben\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\linear_model.py:1673: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


ValueError: Encountered exception in stationarity test ('adf'). This can occur in seasonal settings when a large enough `m` coupled with a large enough `D` difference the training array into too few samples for OLS (input contains 7 samples). Try fitting on a larger training size (raised from LinAlgError: Last 2 dimensions of the array must be square)

In [65]:
arima_model_trim_pesca = pmd.auto_arima(train_trim_pesca["revenue"], start_p=0, start_q=0, start_P=0, start_Q=0, test="adf", seasonal=True,m=4,trace=True)

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(0,0,0)[4]             : AIC=681.628, Time=0.00 sec
 ARIMA(1,2,0)(1,0,0)[4]             : AIC=682.656, Time=0.02 sec
 ARIMA(0,2,1)(0,0,1)[4]             : AIC=684.171, Time=0.02 sec
 ARIMA(0,2,0)(1,0,0)[4]             : AIC=680.391, Time=0.01 sec
 ARIMA(0,2,0)(2,0,0)[4]             : AIC=682.446, Time=0.03 sec
 ARIMA(0,2,0)(1,0,1)[4]             : AIC=682.331, Time=0.02 sec
 ARIMA(0,2,0)(0,0,1)[4]             : AIC=685.449, Time=0.01 sec
 ARIMA(0,2,0)(2,0,1)[4]             : AIC=683.858, Time=0.05 sec
 ARIMA(0,2,1)(1,0,0)[4]             : AIC=683.156, Time=0.02 sec
 ARIMA(1,2,1)(1,0,0)[4]             : AIC=685.152, Time=0.05 sec
 ARIMA(0,2,0)(1,0,0)[4] intercept   : AIC=681.868, Time=0.02 sec

Best model:  ARIMA(0,2,0)(1,0,0)[4]          
Total fit time: 0.271 seconds


In [69]:
arima_model_trim_calcio= pmd.auto_arima(train_trim_calcio["revenue"], start_p=0, start_q=0, start_P=0, start_Q=0, test="adf", seasonal=True,m=2,trace=True)

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[2] intercept   : AIC=750.561, Time=0.00 sec
 ARIMA(1,0,0)(1,1,0)[2] intercept   : AIC=inf, Time=0.05 sec
 ARIMA(0,0,1)(0,1,1)[2] intercept   : AIC=745.971, Time=0.02 sec
 ARIMA(0,0,0)(0,1,0)[2]             : AIC=748.641, Time=0.00 sec
 ARIMA(0,0,1)(0,1,0)[2] intercept   : AIC=748.719, Time=0.01 sec
 ARIMA(0,0,1)(1,1,1)[2] intercept   : AIC=741.862, Time=0.17 sec
 ARIMA(0,0,1)(1,1,0)[2] intercept   : AIC=739.987, Time=0.05 sec
 ARIMA(0,0,1)(2,1,0)[2] intercept   : AIC=741.875, Time=0.08 sec
 ARIMA(0,0,1)(2,1,1)[2] intercept   : AIC=743.670, Time=0.15 sec
 ARIMA(0,0,0)(1,1,0)[2] intercept   : AIC=inf, Time=0.03 sec
 ARIMA(1,0,1)(1,1,0)[2] intercept   : AIC=inf, Time=0.05 sec
 ARIMA(0,0,1)(1,1,0)[2]             : AIC=737.471, Time=0.04 sec
 ARIMA(0,0,1)(0,1,0)[2]             : AIC=746.694, Time=0.01 sec
 ARIMA(0,0,1)(2,1,0)[2]             : AIC=739.423, Time=0.07 sec
 ARIMA(0,0,1)(1,1,1)[2]             : AIC=739.439, Time=0.0

In [5]:
import pickle

In [17]:
with open('./models/arima.pkl', 'wb') as pkl:
    pickle.dump(arima_model_year, pkl)

In [66]:
with open('./models/arima_trim_pesca.pkl', 'wb') as pkl:
    pickle.dump(arima_model_trim_pesca, pkl)

In [61]:
with open('./models/arima_trim_calcio.pkl', 'wb') as pkl:
    pickle.dump(arima_model_trim_calcio, pkl)

Best model (2,0,0)(3,0,1)

In [82]:
test_trim_pesca["SARIMA"] = arima_model_trim_pesca.predict(n_periods=len(test_trim_pesca))
test_trim_pesca.head(10)

,Unnamed: 0,year,revenue,SARIMA
26,26,2020-09-30,1727871.52,4.476301e+06
27,27,2020-12-31,1881887.80,5.787355e+06
28,28,2021-03-31,2258617.01,7.026501e+06
29,29,2021-06-30,1923040.08,7.593583e+06
30,30,2021-09-30,1620445.53,8.131103e+06
31,31,2021-12-31,1704044.85,8.753975e+06
32,32,2022-03-31,1448752.71,9.414009e+06


In [83]:
test_trim_calcio["SARIMA"] = arima_model_trim_calcio.predict(n_periods=len(test_trim_calcio))
test_trim_calcio.head(10)

,Unnamed: 0,year,revenue,SARIMA
26,26,2020-09-30,2254753.87,4.919038e+06
27,27,2020-12-31,1170900.69,1.467532e+06
28,28,2021-03-31,509403.89,8.829062e+05
29,29,2021-06-30,589663.53,4.559836e+05
30,30,2021-09-30,1257807.31,4.918919e+06
31,31,2021-12-31,1350740.40,1.467502e+06
32,32,2022-03-31,781360.53,8.830248e+05


In [84]:
test_year["SARIMA"] = arima_model_year.predict(n_periods=len(test_year))
test_year.head(10)

,Unnamed: 0,year,revenue,SARIMA
7,7.0,2021.0,7506147.47,6.571590e+06
8,8.0,2022.0,1526369.56,6.571590e+06
0,NaN,NaN,NaN,6.571590e+06


In [63]:
def mape(dataset): 
    print(dataset.shape[0])
    sum = np.sum(np.divide(np.abs(dataset["revenue"] - dataset["SARIMA"]),dataset["revenue"] ))
    sum = sum * 100
    return sum/dataset.shape[0]


In [85]:
print(mape(test_year.head(1)))

1
12.450564946258455


In [86]:
print(mape(test_trim_pesca.head(1)))

1
159.06444550590894


In [87]:
print(mape(test_trim_calcio.head(1)))

1
118.16297243962693


In [ ]:
import plotly.express as px
fig = px.line(test, x='data', y=['totale',"ARIMA"])
fig.show()